In [1]:
!pip install -q langchain transformers sentencepiece accelerate bitsandbytes langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 22.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain import HuggingFacePipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"

base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                  load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
    )

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [35]:
llm = HuggingFacePipeline(pipeline=pipe)

In [36]:
ask = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
suggest two ways two lose weight.

Answer:"""

In [37]:
text = llm(ask)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [38]:
print(text.split("Answer:")[1].strip())

Losing weight can be challenging, but there are several strategies that can help. Here are two effective ways to achieve your weight loss goals:

1.  **Dietary Changes**: A significant portion of weight loss comes from making dietary changes. This means focusing on consuming more fruits, vegetables, whole grains, lean proteins, and healthy fats while limiting or avoiding sugary drinks, fast food, and processed snacks. Keeping track of what you eat using a food diary or mobile app can also help identify patterns and make healthier choices. Additionally, incorporating mindful eating practices such as savoring each bite, paying attention to hunger cues, and stopping when satisfied rather than stuffed can lead to reduced calorie intake and improved overall nutrition.

2.  **Regular Exercise**: Regular physical activity plays a crucial role in weight loss by burning calories, building muscle mass, and improving insulin sensitivity. Aim for at least 150 minutes of moderate-intensity aerobic 

# ConversationBufferMemory

In [39]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

prompt = PromptTemplate(
    input_variables=["history", "input"],
    template="""
You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.

{history}
Human: {input}
AI:"""
)

memory = ConversationBufferMemory()

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

In [40]:
output_1 = conversation.predict(input="Can you list 3 places in USA to visit?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [41]:
print(output_1)


You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.


Human: Can you list 3 places in USA to visit?
AI: Here are three popular tourist destinations in the USA:

1. Grand Canyon National Park, Arizona - one of the world's most iconic natural wonders.
2. New Orleans French Quarter, Louisiana - known for its rich history, vibrant music scene, and delicious Creole cuisine.
3. Golden Gate Bridge, California - an engineering marvel with stunning views of San Francisco Bay and the city skyline.


In [42]:
output_2 = conversation.predict(input="Which one has the best weather in winter?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [43]:
print(output_2)


You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.

Human: Can you list 3 places in USA to visit?
AI: 
You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.


Human: Can you list 3 places in USA to visit?
AI: Here are three popular tourist destinations in the USA:

1. Grand Canyon National Park, Arizona - one of the world's most iconic natura

In [44]:
output_3 = conversation.predict(input="Can I visit in winter?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [45]:
print(output_3)


You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.

Human: Can you list 3 places in USA to visit?
AI: 
You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.


Human: Can you list 3 places in USA to visit?
AI: Here are three popular tourist destinations in the USA:

1. Grand Canyon National Park, Arizona - one of the world's most iconic natura

In [46]:
conversation.memory.clear()

# ConversationBufferWindowMemory

In [47]:
from langchain.memory import ConversationBufferWindowMemory

#now the model will onlt remember the last message
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=ConversationBufferWindowMemory( k=1 ),
    verbose=False
)

In [48]:
output_1 = conversation.predict(input="Can you list 3 places in USA to visit?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [49]:
print(output_1)


You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.


Human: Can you list 3 places in USA to visit?
AI: 
1. Grand Canyon (Arizona)
2. Golden Gate Bridge (California)
3. Statue of Liberty (New York)


In [50]:
output_2 = conversation.predict(input="Which one has the best weather in winter?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [51]:
print(output_2)


You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.

Human: Can you list 3 places in USA to visit?
AI: 
You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.


Human: Can you list 3 places in USA to visit?
AI: 
1. Grand Canyon (Arizona)
2. Golden Gate Bridge (California)
3. Statue of Liberty (New York)
Human: Which one has the best weather in 

In [52]:
output_3 = conversation.predict(input="Can I visit in winter?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [53]:
print(output_3)


You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.

Human: Which one has the best weather in winter?
AI: 
You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.

Human: Can you list 3 places in USA to visit?
AI: 
You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand 

In [54]:
conversation.memory.clear()

# ConversationSummaryMemory

In [55]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory

#now the model will summarize the previous chat and bring the important info to the new conversation
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=ConversationSummaryMemory(llm=llm),
    verbose=False
)

In [56]:
output_1 = conversation.predict(input="Can you list 3 places in France to visit?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [57]:
print(output_1)


You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.


Human: Can you list 3 places in France to visit?
AI: 
Here are three popular places to visit in France:
1. The Eiffel Tower (Paris)
2. Palace of Versailles (Versailles)
3. Mont Saint-Michel (Normandy)


In [58]:
output_2 = conversation.predict(input="Which one has the best weather in winter?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [59]:
print(output_2)


You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their f

In [60]:
output_3 = conversation.predict(input="Can I visit in winter?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [61]:
print(output_3)


You are a helpful AI assistant.

Sometimes, I will include previous messages from our chat. If you see them, use them to understand the current question.

If you don’t see any past messages, just treat this as a new conversation.

Just answer the user’s question clearly and directly.
Do not try to manage or summarize the whole conversation. Only respond to the current message.

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their f

In [62]:
conversation.memory.clear()